# 경북대학교 컴퓨터학부 2020117066 김재홍

## 삼성전자 주식을 날짜별로 가지고 와서 차트만들기

## MySql과 연결 (Pymysql)

In [7]:
pip install pymysql

     |████████████████████████████████| 43 kB 1.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### sql에 로컬 사용자 추가, DB권한 부여하기

- 터미널에서 mysql -u root -p로 관리자 로그인
mysql -u root -p

- 로컬 사용자 추가
mysql> CREATE user '사용자'@'localhost' IDENTIFIED BY '사용자_비밀번호';

- DB 권한 부여하기
mysql> GRANT ALL PRIVILEGES ON *.* to '사용자'@'localhost'; # 모든 DB에 접근 가능 *.*
mysql> GRANT ALL PRIVILEGES ON db_name.* to '사용자'@'localhost'; # 특정 DB에만 접근 가능

mysql> FLUSH PRIVILEGES; # 새로운 세팅 적용


### Python으로 Mysql 접속

In [50]:
import pymysql

connection = pymysql.connect(host='localhost', user='jh', password='1234',
                            db='finance', autocommit=True)
# connection 으로부터 cursor 생성
cur = connection.cursor()

In [51]:
# sql문 생성 및 실행 및 fetch
sql = "SELECT * FROM SamsungElec"
#sql = "CREATE TABLE SamsungElec( Price INT NOT NULL, Today_date DATE, PRIMARY KEY (Today_date));"
#sql = "INSERT INTO SamsungElec VALUES (60300, '2022-12-04');"
cur.execute(sql)

# data fetch
rows = cur.fetchall()
print(rows)

# DB연결 종료
# connection.close()

((60300, datetime.date(2022, 12, 4)), (60200, datetime.date(2022, 12, 5)), (59900, datetime.date(2022, 12, 6)))


## Web crawling (requests, BeatifulSoup)
HTML로부터 원하는 정보를 쉽게 가져오도록 비슷한 분류의 데이터별로 나누어주는 라이브러리

In [52]:
import requests
from bs4 import BeautifulSoup

stock_number = { 'SamsungElec' : '005930' }
print(stock_number['SamsungElec'])
url = f"https://finance.naver.com/item/main.naver?code={stock_number['SamsungElec']}"

response = requests.get(url)
notice_page = response.text

soup = BeautifulSoup(notice_page, 'html.parser') #beatifulsoup: HTML 코드의 데이터를 parser로 정리
#print(soup.prettify())

005930


### 원하는 태그를 가져올 수 있도록하는 BeautifulSoup- select 

In [53]:
current_price = soup.select('div.today p.no_today em.no_down span.blind' )
print(current_price)

[<span class="blind">59,100</span>]


In [54]:
for tag in current_price:
    temp = tag.get_text().strip()
print(temp)
temp = temp.replace(',','')
print(temp)
price = int(temp)
print(type(price))

59,100
59100
<class 'int'>


### 오늘 날짜 가져오기 datetime

In [55]:
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
print(type(today))
print(today)

<class 'str'>
2022-12-07


### 오늘 날짜와 현재 가격정보를 저장하기 
insert가 계속 동작하지 않아 원인을 찾아보니 DATE 문법 오류가 뜨고있음.
여기서 python의 datetime을 mysql의 DATE 타입에 저장해야하는데 어려움을 겪고 있음.

sql_save = f"INSERT INTO SamsungElec VALUES ({price}, {today});" 
이렇게 전송할 때 
INSERT INTO SamsungElec VALUES (59900, 2022-12-06);
이런 형식으로 전달되는데 
INSERT INTO SamsungElec VALUES (59900, '2022-12-06');
이렇게 만들어주어야 하기 때문에 
sql_save = f"INSERT INTO SamsungElec VALUES ({price}, '{today}');" 
{today}를 '{today}'로 바꾸어주어 문법 오류를 해결.


In [56]:
sql_save = f"INSERT INTO SamsungElec VALUES ({price}, '{today}');"
print(sql_save)
cur.execute(sql_save)


INSERT INTO SamsungElec VALUES (59100, '2022-12-07');


1

### Insert 되었는지 check

In [57]:
cur.execute(sql)
# rows에 날짜별 가격정보 저장되어있음.
rows = cur.fetchall()
print(rows)

((60300, datetime.date(2022, 12, 4)), (60200, datetime.date(2022, 12, 5)), (59900, datetime.date(2022, 12, 6)), (59100, datetime.date(2022, 12, 7)))


### rows에 저장된 자료형을 dictionary 형태로 저장

In [58]:
# key는 day, value는 price를 저장할 예정임
fin_dict = {}

for i in range (len(rows)):
    # strftime() 을 사용하여 datetime, time 등의 날짜, 시간 형식을 원하는 문자열로 변경
    key = rows[i][1].strftime("%Y-%m-%d")
    value = rows[i][0]
    print("key={}, value={}".format(key, value))
    # dictionary 자료형은 추가할 dic[key]= value 형식으로 추가한다
    fin_dict[key] = value
    
print(fin_dict)

key=2022-12-04, value=60300
key=2022-12-05, value=60200
key=2022-12-06, value=59900
key=2022-12-07, value=59100
{'2022-12-04': 60300, '2022-12-05': 60200, '2022-12-06': 59900, '2022-12-07': 59100}


## plotly를 사용해서 그래프 그리기

In [59]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


### plotly로 그래프 생성하기

In [60]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
fig = go.Figure()
fig.show()

### 그래프에 라인 그리기

In [61]:
print(fin_dict.keys())
print(type(fin_dict.keys()))
# dict.keys()는 자료형이 'dict_keys'이기 때문에 그래프에 사용하기 위해 리스트로 만들어주어야한다.

day_list = list(fin_dict.keys())
price_list = list(fin_dict.values())
print("days={}, prices={}".format(day_list, price_list))

# 그래프 그리기 x축 - day, y축 - price
trace = go.Scatter(x = day_list, y = price_list, mode = 'lines', name = 'lines')
fig.add_trace(trace)
fig.show()




dict_keys(['2022-12-04', '2022-12-05', '2022-12-06', '2022-12-07'])
<class 'dict_keys'>
days=['2022-12-04', '2022-12-05', '2022-12-06', '2022-12-07'], prices=[60300, 60200, 59900, 59100]


### 그래프의 y축 조정하기, 레이아웃 업데이트

In [64]:

# 레이아웃 업데이트
fig.update_layout(go.Layout(title={'text':'삼성전자 주식', 
                                   'font':{'color':'blue', 'size':40}}, # 타이틀
                            xaxis={'title': {'text': '날짜'}, # x축 라벨 추가, 그리드 숨김
                                   'gridwidth':1, 'showgrid':False},
                            yaxis={'title': {'text': '가격'}, # y축 라벨 추가
                                   'gridwidth':1}, # grid line style은 바꿀수 없다.
                            legend ={'borderwidth':2, # 범례 테두리 두께
                                     'bordercolor':'black', # 범례 테두리 색
                                     'font':{'color':'black'} # 범례 글자 색
                                    },
                            paper_bgcolor='#7d7c7c', # 차트 바깥쪽 배경색
                            plot_bgcolor= 'white', # 차트 안쪽 배경색
                            font = {'color':'black'} # 전체 글자(폰트) 색상
                        ))

## y축 값 조정하기
fig.update_yaxes(range=[58000, 62000])